In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from config.dir import DATA_DIR, EM_DATA_DIR ,REPO_DIR
from utils.stimulus_utils import load_textgrids, load_simulated_trfiles
from utils.textgrid import TextGrid
from utils.dsutils import make_semantic_model, make_word_ds, make_phoneme_ds
from utils.npp import zscore, mcorr
import json
from utils.SemanticModel import SemanticModel
from utils.interpdata import lanczosinterp2D

In [ ]:
#! pip3 install numpy pandas voxelwise_tutorials

  Using cached nltools-0.5.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached voxelwise_tutorials-0.2.3.tar.gz (27 kB)
  Preparing metadata (setup.py) ... done
  Using cached nilearn-0.12.0-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of nltools to determine which version is compatible with other requirements. This could take a while.
  Using cached pynv-0.3-py3-none-any.whl.metadata (544 bytes)
  Using cached pydot-4.0.1-py3-none-any.whl.metadata (11 kB)
  Using cached pycortex-1.2.11.tar.gz (37.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached himalaya-0.4.6-py3-none-any.whl.metadata (6.4 kB)
  Using cached pymoten-0.0.8-py3-none-any.whl.metadata (4.1 kB)
  Using cached datalad-1.2.1-py3-none-any.whl.metadata (38 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)

In [ ]:
trim = 5
#ndelays = 4
feature ="eng1000"
subject = 'sub-UTS01'

In [6]:
def get_story_wordseqs(stories):
	grids = load_textgrids(stories, DATA_DIR)
	with open( os.path.join(DATA_DIR, "ds003020/derivative/respdict.json"), "r") as f:
		respdict = json.load(f)
	trfiles = load_simulated_trfiles(respdict)
	wordseqs = make_word_ds(grids, trfiles)
	return wordseqs

In [7]:
with open(os.path.join(EM_DATA_DIR, "sess_to_story.json"), "r") as f:
		sess_to_story = json.load(f) 

In [10]:
sessions = ['10']#,'11']

In [11]:
train_stories, test_stories = [], []
for sess in sessions:
    stories, tstory = sess_to_story[sess][0], sess_to_story[sess][1]
    train_stories.extend(stories)
    if tstory not in test_stories:
        test_stories.append(tstory)
assert len(set(train_stories) & set(test_stories)) == 0, "Train - Test overlap!"


In [12]:
allstories = list(set(train_stories) | set(test_stories))

In [13]:
save_location = os.path.join(REPO_DIR, "results",feature, subject)
print("Saving encoding model & results to:", save_location)
os.makedirs(save_location, exist_ok=True)


Saving encoding model & results to: /Users/genevievelam/Documents/GitHub/stories_fmri/results/eng1000/sub-UTS01


In [14]:
def downsample_word_vectors(stories, word_vectors, wordseqs):
	"""Get Lanczos downsampled word_vectors for specified stories.

	Args:
		stories: List of stories to obtain vectors for.
		word_vectors: Dictionary of {story: <float32>[num_story_words, vector_size]}

	Returns:
		Dictionary of {story: downsampled vectors}
	"""
	downsampled_semanticseqs = dict()
	for story in stories:
		downsampled_semanticseqs[story] = lanczosinterp2D(
			word_vectors[story], wordseqs[story].data_times, 
			wordseqs[story].tr_times, window=3)
	return downsampled_semanticseqs

In [15]:
def get_eng1000_vectors(allstories):
	"""Get Eng1000 vectors (985-d) for specified stories.

	Args:
		allstories: List of stories to obtain vectors for.

	Returns:
		Dictionary of {story: downsampled vectors}
	"""
	eng1000 = SemanticModel.load(os.path.join(EM_DATA_DIR, "english1000sm.hf5"))
	wordseqs = get_story_wordseqs(allstories)
	vectors = {}
	for story in allstories:
		sm = make_semantic_model(wordseqs[story], [eng1000], [985])
		vectors[story] = sm.data
	return downsample_word_vectors(allstories, vectors, wordseqs)

In [16]:
_FEATURE_CONFIG = {
	#"articulation": get_articulation_vectors,
	#"phonemerate": get_phonemerate_vectors,
	#"wordrate": get_wordrate_vectors,
	"eng1000": get_eng1000_vectors,
}

In [17]:
def get_feature_space(feature, *args):
	return _FEATURE_CONFIG[feature](*args)

In [ ]:
downsampled_feat = get_feature_space(feature, allstories)
print("Stimulus & Response parameters:")
#print("trim: %d, ndelays: %d" % (trim, ndelays))

Stimulus & Response parameters:
trim: 5, ndelays: 4


In [49]:
def convert_to_serializable(downsampled_feat):
    """Convert downsampled feature dictionary to a serializable format."""
    
    serializable_dict = downsampled_feat.tolist()

    return serializable_dict


In [ ]:
def apply_zscore_and_hrf(stories, downsampled_feat, trim):
	"""Get (z-scored and delayed) stimulus for train and test stories.
	The stimulus matrix is delayed (typically by 2,4,6,8 secs) to estimate the
	hemodynamic response function with a Finite Impulse Response model.

	Args:
		stories: List of stimuli stories.

	Variables:
		downsampled_feat (dict): Downsampled feature vectors for all stories.
		trim: Trim downsampled stimulus matrix.
		delays: List of delays for Finite Impulse Response (FIR) model.

	Returns:
		delstim: <float32>[TRs, features * ndelays]
	"""
	stim = [zscore(downsampled_feat[s][5+trim:-trim]) for s in stories]
	stim = np.vstack(stim)
	#delays = range(1, ndelays+1)
	#delstim = make_delayed(stim, delays)
	return stim#delstim

In [ ]:
delRstim = apply_zscore_and_hrf(train_stories, downsampled_feat, trim)

In [52]:
delRstim.shape

(1824, 985)

In [34]:
import pathlib	
import h5py
def get_response(stories, subject):
	"""Get the subject"s fMRI response for stories."""
	#main_path = pathlib.Path(__file__).parent.parent.resolve()
	subject_x = subject.split('-')[1]
	subject_dir = os.path.join(DATA_DIR, "ds003020/derivative/preprocessed_data/%s" % subject_x)
	base = subject_dir
	resp = []
	for story in stories:
		resp_path = os.path.join(base, "%s.hf5" % story)
		hf = h5py.File(resp_path, "r")
		resp.extend(hf["data"][:])
		hf.close()
	return np.array(resp)

In [40]:
# Response
zRresp = get_response(train_stories, subject)
print("zRresp: ", zRresp.shape)


zRresp:  (1824, 81126)


In [57]:
# Response
zRresp = get_response(train_stories, 'sub-UTS02')
print("zRresp: ", zRresp.shape)


zRresp:  (1824, 94251)


In [ ]:
#save_location = os.path.join(REPO_DIR, "results",feature, 'sub-UTS03')
#print("Saving encoding model & results to:", save_location)
#os.makedirs(save_location, exist_ok=True)

#with open(save_location+'/fmri.json', "w") as file:
#        json.dump(convert_to_serializable(zRresp),file, indent=4)

Saving encoding model & results to: /Users/genevievelam/Documents/GitHub/stories_fmri/results/eng1000/sub-UTS03


# Save feature 

In [ ]:
#with open(save_location+'/features.json', "w") as file:
#        json.dump(convert_to_serializable(delRstim),file, indent=4)

In [ ]:
#with open(save_location+'/fmri.json', "w") as file:
#        json.dump(convert_to_serializable(zRresp),file, indent=4)